In [4]:
#Import Libraries
import os,json
import time
import pandas as pd
import numpy as np
from pandas.io.json import json_normalize

### Convert 58k+ JSON to 30 CSV files (Each file with 2000 records)

In [3]:
#Load data
start_time = time.time()

path_main = 'Parkinsons_mPower_Voice_Features/'
folder = 'Folder '
for i in range (1,31):
    folder_name = folder + str(i)
    path_to_json = path_main+folder_name

#path_to_json = 'Parkinsons_mPower_Voice_Features/Folder 30'
    json_files = [pos_json for pos_json in os.listdir(path_to_json) if pos_json.endswith('.json')]
    pd_list= []
    for index, js in enumerate(json_files):
        with open(os.path.join(path_to_json, js)) as json_file:
            json_dict = json.load(json_file)
            pd_list.append(json_dict)
    pd_nor = json_normalize(pd_list)
    print("--- %s seconds ---" % (time.time() - start_time))


    #Verifying the number of files loaded
    len(pd_list)

    #Create multiple columns for MFCC and chroma_vector
    MFCC = pd_nor['features.audio.MFCC'].apply(pd.Series)
    MFCC = MFCC.rename(columns = lambda x : 'MFCC-' + str(x))

    chroma_vector = pd_nor['features.audio.chroma_vector'].apply(pd.Series)
    chroma_vector = chroma_vector.rename(columns = lambda x : 'chroma-vector-' + str(x))

    #Concatenate the new columns to the main dataframe
    pd_nor = pd.concat([pd_nor[:], MFCC[:], chroma_vector[:]], axis=1)

    #Drop MFCC & chroma vector columns
    pd_nor.drop(['features.audio.MFCC'], axis=1, inplace=True)
    pd_nor.drop(['features.audio.chroma_vector'], axis=1, inplace=True)


    df = pd.DataFrame(pd_nor)

    #Retain columns required
    df1 = df.drop(['appversion','audiocountdown','createdon','healthcode','medtimepoint','recordID2','phoneinfo','recordingID','unnamed'], axis = 1)

    #Extract column names
    column_names = list(df1.columns.values)

    #Empty dataframe
    new_df = pd.DataFrame()

    #Calulate var for each feature
    for j in range(len(df1.columns)):
        temp = ''.join(column_names[j].split('.',2)[-1])
        new_col_name = temp + '_' + 'var'
        temp = df1.iloc[:,j].apply(pd.Series)
        new_df[new_col_name] = temp.var(axis=1, skipna=True)

    # Add columns back to to the data set
    result = pd.concat([df[['appversion','audiocountdown','createdon','healthcode','medtimepoint','recordID2','phoneinfo','recordingID','unnamed']], new_df], axis=1, join_axes=[df.index])
    result.columns.values

    #Create a csv file - to avoid reprocessing of all json files
    df = pd.DataFrame(result)
    filename = 'pd-'+str(i)+'.csv'
    df.to_csv(filename, index=False)

--- 147.83627128601074 seconds ---
--- 333.9961552619934 seconds ---
--- 478.7590091228485 seconds ---
--- 619.8587391376495 seconds ---
--- 762.5776979923248 seconds ---
--- 904.6566708087921 seconds ---
--- 1039.9115023612976 seconds ---
--- 1177.2814173698425 seconds ---
--- 1347.006910085678 seconds ---
--- 1503.8823132514954 seconds ---
--- 1646.3528242111206 seconds ---
--- 1804.1526987552643 seconds ---
--- 1980.0521972179413 seconds ---
--- 2162.800525188446 seconds ---
--- 2310.84694647789 seconds ---
--- 2450.8800477981567 seconds ---
--- 2588.80268740654 seconds ---
--- 2729.2654750347137 seconds ---
--- 2875.1180596351624 seconds ---
--- 3020.5594053268433 seconds ---
--- 3167.094987154007 seconds ---
--- 3313.989399909973 seconds ---
--- 3459.6679422855377 seconds ---
--- 3626.0027158260345 seconds ---
--- 3780.8816409111023 seconds ---
--- 3926.7254400253296 seconds ---
--- 4075.389240026474 seconds ---
--- 4223.158165216446 seconds ---
--- 4362.234123706818 seconds ---
-

### Join demographic and csv files with mean, var and std

In [13]:
#Read demographics excel
df_dem = pd.read_excel('Demographics.xlsx')
print(df_dem.shape)

(6805, 33)


In [14]:
#Drop unnecessary columns
df_dem = df_dem.drop(['appVersion','phoneInfo'], axis = 1)
print(df_dem.shape)

(6805, 31)


In [15]:
#Read CSV of mean, var & std values of PD audio features
PD_mean = pd.read_csv('PD_audio_features_mean.csv')
print(PD_mean.shape)
PD_var = pd.read_csv('PD_audio_features_var.csv')
print(PD_var.shape)
PD_std = pd.read_csv('PD_audio_features_std.csv')
print(PD_std.shape)

(58053, 43)
(58053, 43)
(58053, 43)


In [16]:
#Number of unique participants in the voice activity
# Column : healthcode
print('Unique participants in the demographics dataset: %d' %(df_dem.healthCode.nunique()))

print('Unique participants in the voice activity: %d' %(PD_mean.healthcode.nunique()))

#Are they duplicate entries in demographics data? -- No
dup_participant = df_dem.groupby('healthCode')
dup_participant.filter(lambda x: len(x) > 1)

Unique participants in the demographics dataset: 6805
Unique participants in the voice activity: 5642


,ROW_ID,ROW_VERSION,recordId,healthCode,createdOn,age,are-caretaker,deep-brain-stimulation,diagnosis-year,education,...,packs-per-day,past-participation,phone-usage,professional-diagnosis,race,smartphone,smoked,surgery,video-usage,years-smoking


In [17]:
#Rename column name in demographics data
df_dem.rename(columns={'healthCode': 'healthcode'}, inplace = True)

In [18]:
#Merge demographic and PD audio features(mean) dataset
result_mean = pd.merge(PD_mean, df_dem, on = ['healthcode'])
print(result_mean.shape)
print(result_mean.columns.values)

#Merge demographic and PD audio features(var) dataset
result_var = pd.merge(PD_var, df_dem, on = ['healthcode'])
print(result_var.shape)
print(result_var.columns.values)

#Merge demographic and PD audio features(std) dataset
result_std = pd.merge(PD_std, df_dem, on = ['healthcode'])
print(result_std.shape)
print(result_std.columns.values)

(56444, 73)
['appversion' 'audiocountdown' 'createdon' 'healthcode' 'medtimepoint'
 'recordID2' 'phoneinfo' 'recordingID' 'unnamed' 'ZCR_mean'
 'chroma_deviation_mean' 'energy_mean' 'entropy_mean'
 'spectral_centroid_mean' 'spectral_entropy_mean' 'spectral_flux_mean'
 'spectral_rolloff_mean' 'spectral_spread_mean' 'MFCC-0_mean'
 'MFCC-1_mean' 'MFCC-2_mean' 'MFCC-3_mean' 'MFCC-4_mean' 'MFCC-5_mean'
 'MFCC-6_mean' 'MFCC-7_mean' 'MFCC-8_mean' 'MFCC-9_mean' 'MFCC-10_mean'
 'MFCC-11_mean' 'MFCC-12_mean' 'chroma-vector-0_mean'
 'chroma-vector-1_mean' 'chroma-vector-2_mean' 'chroma-vector-3_mean'
 'chroma-vector-4_mean' 'chroma-vector-5_mean' 'chroma-vector-6_mean'
 'chroma-vector-7_mean' 'chroma-vector-8_mean' 'chroma-vector-9_mean'
 'chroma-vector-10_mean' 'chroma-vector-11_mean' 'ROW_ID' 'ROW_VERSION'
 'recordId' 'createdOn' 'age' 'are-caretaker' 'deep-brain-stimulation'
 'diagnosis-year' 'education' 'employment' 'gender' 'health-history'
 'healthcare-provider' 'home-usage' 'last-smoked' '

In [1]:
#Create new csv file
result_mean.to_csv('PD_audio_features_mean_with_demographics.csv', index = False)

result_var.to_csv('PD_audio_features_var_with_demographics.csv', index = False)

result_std.to_csv('PD_audio_features_std_with_demographics.csv', index = False)

NameError: name 'result_mean' is not defined

### Generate case-control data

In [5]:
#Read file
result_mean = pd.read_csv('PD_audio_features_mean_with_demographics.csv')
print(result_mean.shape)

result_var = pd.read_csv('PD_audio_features_var_with_demographics.csv')
print(result_var.shape)

result_std = pd.read_csv('PD_audio_features_std_with_demographics.csv')
print(result_std.shape)

(56444, 73)
(56444, 73)
(56444, 73)


In [6]:
#Generate case-control data set for mean

#Create a match pair column
result_mean['match_pair'] = 0

#Extract indexes of cases - PWP
before_PD = result_mean.index[result_mean['medtimepoint'] == 'Immediately before Parkinson medication'].tolist()

for i in before_PD:
    temp_age = result_mean.iloc[i]['age']
    temp_gender = result_mean.iloc[i]['gender']
    temp = result_mean[(result_mean['age'] == temp_age) 
                       & (result_mean['gender'] == temp_gender) 
                       & (result_mean['medtimepoint'] == "I don't take Parkinson medications") 
                       & (result_mean['match_pair'] == 0)]
  
    #To pick cases where atleast 2 controls are found
    if temp.shape[0] > 1:
        result_mean.match_pair.iloc[i] = i
        result_mean.match_pair.iloc[temp.sample(n=2).index.tolist()] = i

C:\Users\vanlo\Anaconda3\lib\site-packages\pandas\core\indexing.py:141: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [7]:
#Find columns with NA values
result_mean.isnull().any()
print(result_mean.shape)
result_mean = result_mean[result_mean['chroma_deviation_mean'].notnull()]
result_mean = result_mean[result_mean['medtimepoint'].notnull()]
print(result_mean.shape)

(56444, 74)
(56233, 74)


In [8]:
#Generate case-control data set for var

#Create a match pair column
result_var['match_pair'] = 0

#Extract indexes of cases - PWP
before_PD = result_var.index[result_var['medtimepoint'] == 'Immediately before Parkinson medication'].tolist()

for i in before_PD:
    temp_age = result_var.iloc[i]['age']
    temp_gender = result_var.iloc[i]['gender']
    temp = result_var[(result_var['age'] == temp_age) 
                       & (result_var['gender'] == temp_gender) 
                       & (result_var['medtimepoint'] == "I don't take Parkinson medications") 
                       & (result_var['match_pair'] == 0)]
  
    #To pick cases where atleast 2 controls are found
    if temp.shape[0] > 1:
        result_var.match_pair.iloc[i] = i
        result_var.match_pair.iloc[temp.sample(n=2).index.tolist()] = i

C:\Users\vanlo\Anaconda3\lib\site-packages\pandas\core\indexing.py:141: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [15]:
#Find columns with NA values
result_var.isnull().any()
print(result_var.shape)
result_var = result_var[result_var['medtimepoint'].notnull()]
result_var = result_var[result_var['chroma_deviation_var'].notnull()]
print(result_var.shape)

(56233, 74)
(56233, 74)


In [18]:
#Generate case-control data set for mean

#Create a match pair column
result_std['match_pair'] = 0

#Extract indexes of cases - PWP
before_PD = result_std.index[result_std['medtimepoint'] == 'Immediately before Parkinson medication'].tolist()

for i in before_PD[:500]:
    temp_age = result_std.iloc[i]['age']
    temp_gender = result_std.iloc[i]['gender']
    temp = result_std[(result_std['age'] == temp_age) 
                       & (result_std['gender'] == temp_gender) 
                       & (result_std['medtimepoint'] == "I don't take Parkinson medications") 
                       & (result_std['match_pair'] == 0)]
  
    #To pick cases where atleast 2 controls are found
    if temp.shape[0] > 1:
        result_std.match_pair.iloc[i] = i
        result_std.match_pair.iloc[temp.sample(n=2).index.tolist()] = i

C:\Users\vanlo\Anaconda3\lib\site-packages\pandas\core\indexing.py:141: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [22]:
#Find columns with NA values
result_std.isnull().any()
print(result_std.shape)
result_std = result_std[result_std['medtimepoint'].notnull()]
result_std = result_std[result_std['chroma_deviation_std'].notnull()]
result_std = result_std[result_std['spectral_centroid_std'].notnull()]
result_std = result_std[result_std['spectral_spread_std'].notnull()]
print(result_std.shape)

(56444, 74)
(56233, 74)


In [23]:
result_mean.to_csv('PD_mean_case_control.csv')
result_var.to_csv('PD_var_case_control.csv')
result_std.to_csv('PD_std_case_control.csv')